In [2]:
import numpy as np
from scipy.interpolate import interp2d, interp1d , RegularGridInterpolator
from scipy.integrate import quad
from scipy.interpolate import RegularGridInterpolator
import matplotlib.pyplot as plt

In [3]:
data=np.loadtxt('./dn_dM.txt',skiprows=1)
M_list=(data[:,0])
z_list=(data[:,1])
dndzM_list=(data[:,2])

In [4]:
import pandas as pd

data = pd.read_csv('./dn_dM.txt', sep='\s+', header=0)
M_list = data[data.columns[0]].values
z_list = data[data.columns[1]].values
dndzM_list = data[data.columns[2]].values
M_test, z_test = 1e10, 5
if M_test < M_list.min() or M_test > M_list.max() or z_test < z_list.min() or z_test > z_list.max():
    print(f"Error: M={M_test}, z={z_test} is out of data range.")
else:
    print(f_dndzM(M_test, z_test))

Error: M=10000000000.0, z=5 is out of data range.


In [5]:
# 检查输入数据
print(f"M_list: {M_list}")
print(f"z_list: {z_list}")

# 检查 x 和 y 坐标是否单调递增
print(f"Is M_list monotonically increasing? {np.all(np.diff(M_list) > 0)}")
print(f"Is z_list monotonically increasing? {np.all(np.diff(z_list) > 0)}")

# 检查 x 和 y 坐标是否有重复值
print(f"Are there any duplicate values in M_list? {len(np.unique(M_list)) != len(M_list)}")
print(f"Are there any duplicate values in z_list? {len(np.unique(z_list)) != len(z_list)}")

M_list: [ 4.  4.  4. ... 40. 40. 40.]
z_list: [1.00000000e+05 1.02329299e+05 1.04712855e+05 ... 9.54992586e+14
 9.77237221e+14 1.00000000e+15]
Is M_list monotonically increasing? False
Is z_list monotonically increasing? False
Are there any duplicate values in M_list? True
Are there any duplicate values in z_list? True


In [6]:
# 对数据进行预处理
M_list_unique = np.unique(M_list)
z_list_unique = np.unique(z_list)

# 在新的网格上进行二维插值
f_dndzM = interp2d(M_list_unique, z_list_unique, dndzM_list.reshape(len(z_list_unique), len(M_list_unique)), kind='linear')

/tmp/ipykernel_810/3151119667.py:6: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  f_dndzM = interp2d(M_list_unique, z_list_unique, dndzM_list.reshape(len(z_list_unique), len(M_list_unique)), kind='linear')


In [7]:
# 计算任意 M, z 时的质量函数
M_test = 5
z_test = 222000
dndzM_interp = f_dndzM(M_test, z_test)
print(dndzM_interp)

[1.32786275e-07]


/tmp/ipykernel_810/4049118612.py:4: DeprecationWarning:         `interp2d` is deprecated!
        `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.13.0.

        For legacy code, nearly bug-for-bug compatible replacements are
        `RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
        scattered 2D data.

        In new code, for regular grids use `RegularGridInterpolator` instead.
        For scattered data, prefer `LinearNDInterpolator` or
        `CloughTocher2DInterpolator`.

        For more details see
        `https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  dndzM_interp = f_dndzM(M_test, z_test)


In [8]:
Mmin = lambda z: 2.8e9 * (1+z)**(-3/2) # M_sun
Omega_m = 0.32
rho_c = 2.7752e11 * 0.6774**2 # M_sun/Mpc^3

def fcoll(z):
    return (1/rho_c) * quad(lambda M: M * dndzM_interp(M, z), Mmin(z), np.inf)[0]

In [9]:
import scipy.misc as misc

def dfcoll_dz(z):
    return misc.derivative(fcoll, z, dx=1e-3)

def H(z):
    return 67.72 * np.sqrt(0.32*(1+z)**3 + 0.68) # km/s/Mpc
    
def SFRD(z):
    return 0.1 * (0.048/0.32) * rho_c * dfcoll_dz(z) / H(z)

In [10]:
data=np.loadtxt('./SED.txt',skiprows=1)
nu_list=(data[:,0])
s_list=(data[:,1])

# 创建一维插值函数
f_s = interp1d(nu_list, s_list, kind='linear')

# 计算任意频率 ν 的 s(ν)
nu_test = 2.7e16
s_interp = f_s(nu_test)
print(s_interp)

2.9626581462443565e+23


In [17]:
def J(nu):
    integrand = lambda z: s_interp(nu/(1+z)) * SFRD(z) / (4*np.pi * H(z) * (1+z))
    return quad(integrand, 5, 30)[0]

# 计算 J(ν) 在 0.1-10 μm 范围内的值
wavelength = np.logspace(-1, 1, 100) # μm
print(wavelength)
nu = 3e8 / (wavelength * 1e-6) # Hz
J_nu = [J(n) for n in nu]
print(J(nu))

import matplotlib.pyplot as plt
plt.plot(wavelength, J(nu))
plt.xlabel(r"Wavelength $\lambda$ [$\mu$m]")
plt.ylabel(r"$J(\nu)$ [erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$ Hz$^{-1}$]")
plt.show()


[ 0.1         0.10476158  0.10974988  0.1149757   0.12045035  0.12618569
  0.13219411  0.13848864  0.14508288  0.15199111  0.15922828  0.16681005
  0.17475284  0.18307383  0.19179103  0.2009233   0.21049041  0.22051307
  0.23101297  0.24201283  0.25353645  0.26560878  0.27825594  0.29150531
  0.30538555  0.31992671  0.33516027  0.35111917  0.36783798  0.38535286
  0.40370173  0.42292429  0.44306215  0.46415888  0.48626016  0.5094138
  0.53366992  0.55908102  0.58570208  0.61359073  0.64280731  0.67341507
  0.70548023  0.7390722   0.77426368  0.81113083  0.84975344  0.89021509
  0.93260335  0.97700996  1.02353102  1.07226722  1.12332403  1.17681195
  1.23284674  1.29154967  1.35304777  1.41747416  1.48496826  1.55567614
  1.62975083  1.70735265  1.78864953  1.87381742  1.96304065  2.05651231
  2.15443469  2.25701972  2.36448941  2.47707636  2.59502421  2.71858824
  2.84803587  2.98364724  3.12571585  3.27454916  3.43046929  3.59381366
  3.76493581  3.94420606  4.1320124   4.32876128  4.

TypeError: 'numpy.ndarray' object is not callable